In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.DSSM.get_default_preprocessor()
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 5561.93it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:09<00:00, 1930.82it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 2244732.58it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 8292.47it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:08<00:00, 2332.45it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:13<00:00, 1400.26it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 1

In [4]:
preprocessor.context

{'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f033a9b2e48>,
 'vocab_size': 9645,
 'embedding_input_dim': 9645}

In [5]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=1,
    num_neg=4
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
)

In [6]:
padding_callback = mz.models.DSSM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=32,
    stage='train',
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=32,
    stage='dev',
    callback=padding_callback
)

In [7]:
model = mz.models.DSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['vocab_size']
model.params['mlp_num_layers'] = 3
model.params['mlp_num_units'] = 300
model.params['mlp_num_fan_out'] = 128
model.params['mlp_activation_func'] = 'relu'

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

DSSM(
  (mlp_left): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9645, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (mlp_right): Sequential(
    (0): Sequential(
      (0): Linear(in_features=9645, out_features=300, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (2): Sequential(
      (0): Linear(in_features=300, out_features=300, bias=True)
      (1): ReLU()
    )
    (3): Sequential(
      (0): Linear(in_features=300, out_features=128, bias=True)
      (1): ReLU()
    )
  )
  (out): Linear(in_features=1, out_features=1, bias=True)
)

In [8]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-32 Loss-1.597]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3975 - normalized_discounted_cumulative_gain@5(0.0): 0.4874 - mean_average_precision(0.0): 0.435



[Iter-64 Loss-1.565]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4605 - normalized_discounted_cumulative_gain@5(0.0): 0.54 - mean_average_precision(0.0): 0.4968



[Iter-96 Loss-1.539]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4115 - normalized_discounted_cumulative_gain@5(0.0): 0.4961 - mean_average_precision(0.0): 0.4459



[Iter-128 Loss-1.515]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4154 - normalized_discounted_cumulative_gain@5(0.0): 0.4923 - mean_average_precision(0.0): 0.4532



[Iter-160 Loss-1.496]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4442 - normalized_discounted_cumulative_gain@5(0.0): 0.5179 - mean_average_precision(0.0): 0.4803



[Iter-192 Loss-1.472]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4116 - normalized_discounted_cumulative_gain@5(0.0): 0.4843 - mean_average_precision(0.0): 0.4489



[Iter-224 Loss-1.447]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4345 - normalized_discounted_cumulative_gain@5(0.0): 0.5089 - mean_average_precision(0.0): 0.4745



[Iter-256 Loss-1.409]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3925 - normalized_discounted_cumulative_gain@5(0.0): 0.4835 - mean_average_precision(0.0): 0.437



[Iter-288 Loss-1.389]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3771 - normalized_discounted_cumulative_gain@5(0.0): 0.4633 - mean_average_precision(0.0): 0.4219



[Iter-320 Loss-1.362]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.455 - normalized_discounted_cumulative_gain@5(0.0): 0.5211 - mean_average_precision(0.0): 0.4812

Cost time: 2646.1402447223663s
